# Testing for Heteroscedasticity

## Data set

We will be using the same modified wages data set of the previous
section.

Remember that this data has the following variables:

-   `wage`: average hourly earnings
-   `educ`: years of education
-   `exper`: years potential experience
-   `tenure`: years with current employer
-   `married`: =1 if married
-   `numdep`: number of dependents
-   `gender`: male or female
-   `skin`: color of the skin (white or nonwhite)

## Base regression

In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

data = pd.read_csv("./data/wage.csv")
data.head()

As before, we first want to construct the dummy variables for `gender`
and `skin`:

In [2]:
data = pd.get_dummies(data, columns=["gender", "skin"], drop_first=False, dtype=int)
data.head()

Now, let’s run a regression:

In [3]:
model = smf.ols("wage ~ educ + exper + tenure + married + numdep + gender_female + skin_nonwhite", data=data)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                   wage   R-squared:                       0.370
Model:                            OLS   Adj. R-squared:                  0.362
Method:                 Least Squares   F-statistic:                     43.54
Date:                Sun, 03 Nov 2024   Prob (F-statistic):           2.53e-48
Time:                        22:41:31   Log-Likelihood:                -1311.4
No. Observations:                 526   AIC:                             2639.
Df Residuals:                     518   BIC:                             2673.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -1.9799      0.783     -2.530

## Performing the White test

Now, let’s perform the White test, which is also available in the
`statsmodels` library. **Note the following**:

-   We use the same independent variables of the original model,
    obtained with `results.model.exog`.
-   The estimated residual are obtained from the previous estimation
    with `results.resid`.
-   `statsmodels` automatically include squares and interaction terms to
    run the auxiliary regression for the White test (so we don’t have to
    do this ourselves).

In [4]:
import statsmodels.stats.diagnostic as diag

white_test = diag.het_white(results.resid, results.model.exog)

# The output consists of the test statistic and the associated p-value
labels = ['Test Statistic', 'Test Statistic p-value', 'F-Statistic', 'F-Statistic p-value']
white_test_summary = dict(zip(labels, white_test))

for key, value in white_test_summary.items():
    print(f"{key}: {value}")

Test Statistic: 76.30870031228504
Test Statistic p-value: 1.7487062379433416e-05
F-Statistic: 2.6143065587494148
F-Statistic p-value: 6.246658985394247e-06

### Analyze the results of the initial White test

-   Observe that `p-values` from the White test are close to zero
-   Given that the null hypothesis in the White Test is that residuals
    are homoscedastic, in this case, **we reject the null hypothesis**,
    therefore we say that **residuals are heteroscedastic instead**,
    which is a problem and we should try to correct for this.

## Detect which variables affect the variance of the error term $\text{Var}(u)$

To accomplish this, we want to see which terms are significantly
affecting the estimated squared error term (i.e., $\widehat{u}^2$).
Therefore, we need to run the regression of the White test ourselves (or
something similar to it).

For example, we regress $\widehat{u}^2$ on the same independent
variables and their squares, but excluding the dummy variables and the
interaction between variables for simplicity:

In [5]:
# Calculate the squared residuals
data['resid_sq'] = results.resid ** 2

# Calculate the square of some of the independent variables

data['educ_sq'] = data['educ'] ** 2
data['exper_sq'] = data['exper'] ** 2
data['tenure_sq'] = data['tenure'] ** 2
data['numdep_sq'] = data['numdep'] ** 2

# Fit the auxiliary regression
aux_model = smf.ols('resid_sq ~ educ + exper + tenure + married + numdep + gender_female + skin_nonwhite + educ_sq + exper_sq + tenure_sq + numdep_sq ', data=data)
aux_results = aux_model.fit()

# Display the auxiliary regression results
print("\nAuxiliary Regression Summary (for White's Test):\n")
print(aux_results.summary())


Auxiliary Regression Summary (for White's Test):

                            OLS Regression Results                            
Dep. Variable:               resid_sq   R-squared:                       0.119
Model:                            OLS   Adj. R-squared:                  0.101
Method:                 Least Squares   F-statistic:                     6.337
Date:                Sun, 03 Nov 2024   Prob (F-statistic):           7.20e-10
Time:                        22:41:31   Log-Likelihood:                -2343.6
No. Observations:                 526   AIC:                             4711.
Df Residuals:                     514   BIC:                             4762.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------

### Analyze the results of the White test regression

-   We can see that some of the variables are significant in explaining
    the square of the estimated residuals
-   In particular, the square of `educ` and `exper` are significant, so
    we will use them to try to correct for heteroscedasticity.

## Run a WLS estimation

To try fixing the problem of heteroscedasticity, lets run a WLS
estimation:

We will estimate the basic model again with OLS and the modified model
with Weighted Least Squares or WLS, for clarity:

In [6]:
# Load the dataset and construct dummies
data = pd.read_csv("data/wage.csv")
data = pd.get_dummies(data, columns=["gender", "skin"], drop_first=False, dtype=int)

# Initial OLS model
model_ols = smf.ols('wage ~ educ + exper + tenure + married + numdep + gender_female + skin_nonwhite', data=data)
results_ols = model_ols.fit()

# Estimate residuals
residuals = results_ols.resid

# Assuming variance is proportional to the square of 'educ' and 'exper' as concluded above
data['h'] = data['educ'] ** 2 + data['exper'] ** 2

# Calculate the inverse of the root of h
weights = 1 / np.sqrt(data['h'])

# Create weighted variables by multiplying each variable by the weights
data['w_wage'] = data['wage'] * weights
data['w_educ'] = data['educ'] * weights
data['w_exper'] = data['exper'] * weights
data['w_tenure'] = data['tenure'] * weights
data['w_married'] = data['married'] * weights
data['w_numdep'] = data['numdep'] * weights
data['w_gender_female'] = data['gender_female'] * weights
data['w_skin_nonwhite'] = data['skin_nonwhite'] * weights

# Run OLS with weighted variables (that is, WLS)
model_wls = smf.ols('w_wage ~ w_educ + w_exper + w_tenure + w_married + w_numdep + w_gender_female + w_skin_nonwhite', data=data)

results_wls = model_wls.fit()

# Output the results
print("\nWLS model summary:\n")
print(results_wls.summary())


WLS model summary:

                            OLS Regression Results                            
Dep. Variable:                 w_wage   R-squared:                       0.382
Model:                            OLS   Adj. R-squared:                  0.373
Method:                 Least Squares   F-statistic:                     45.71
Date:                Sun, 03 Nov 2024   Prob (F-statistic):           2.37e-50
Time:                        22:41:31   Log-Likelihood:                 316.96
No. Observations:                 526   AIC:                            -617.9
Df Residuals:                     518   BIC:                            -583.8
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -0

Now, we perform the White test once again to check whether errors are
now homoscedastic (which is what we want) or are still heteroscedastic:

In [7]:
white_test = diag.het_white(results_wls.resid, results_wls.model.exog)

# The output consists of the test statistic and the associated p-value
labels = ['Test Statistic', 'Test Statistic p-value', 'F-Statistic', 'F-Statistic p-value']
white_test_summary = dict(zip(labels, white_test))

for key, value in white_test_summary.items():
    print(f"{key}: {value}")

Test Statistic: 42.91062121627159
Test Statistic p-value: 0.140575269122546
F-Statistic: 1.282743692289221
F-Statistic p-value: 0.13561295323905587

As we can see, the `p-value` of the White test is larger than 0.1:

    Test Statistic p-value: 0.140575269122546

This means that we **fail to reject the null hypothesis of the White
test, and so we consider the residuals to be homoscedastic**.

**Conclusion**: we have managed to detect and correct for
heteroscedasticity!